In [1]:
from qat.fermion.transforms import (get_jw_code, recode_integer)
from qubit_pool import QubitPool
from molecule_factory import MoleculeFactory
from get_energy_ucc import EnergyUCC

qubitpool = QubitPool()
molecule_factory = MoleculeFactory()
energy_ucc = EnergyUCC()

In [2]:
molecule_symbol = 'H4'
type_of_generator = 'sUPCCGSD'
transform = 'JW'
# the user type "active = True" the active space selection case
# Here user is obliged to check the thresholds epsilon_1 and epsilon_2  inserted in "MoleculeFactory" to select: the active electorns and active orbitals
active = True

In [3]:
r, geometry, charge, spin, basis = molecule_factory.get_parameters(molecule_symbol)
print(" --------------------------------------------------------------------------")
print("Running in the active case: ")
print("                     molecule symbol: %s " %(molecule_symbol))
print("                     molecule basis: %s " %(basis))
print("                     type of generator: %s " %(type_of_generator))
print("                     transform: %s " %(transform))
print(" --------------------------------------------------------------------------")

print(" --------------------------------------------------------------------------")
print("                                                          ")
print("                      Generate Hamiltonians and Properties from :")
print("                                                          ")
print(" --------------------------------------------------------------------------")
print("                                                          ")

hamiltonian, hamiltonian_sp, n_elec, noons_full, orb_energies_full, info = molecule_factory.generate_hamiltonian(molecule_symbol, active=active, transform=transform)

print(" --------------------------------------------------------------------------")
print("                                                          ")
print("                      Generate Cluster OPS from :")
print("                                                          ")
print(" --------------------------------------------------------------------------")
print("                                                          ")

# for UCCSD
# pool_size,cluster_ops,cluster_ops_sp, theta_MP2, hf_init = molecule_factory.generate_cluster_ops(molecule_symbol, type_of_generator=type_of_generator, 
#                                transform=transform, active=active)
# hf_init_sp = recode_integer(hf_init, get_jw_code(hamiltonian_sp.nbqbits))


# for other types of generators
pool_size,cluster_ops,cluster_ops_sp =molecule_factory.generate_cluster_ops(molecule_symbol, 
                    type_of_generator=type_of_generator, transform=transform, active=active)
hf_init = molecule_factory.find_hf_init(hamiltonian, n_elec, noons_full, orb_energies_full)
nbqbits = hamiltonian_sp.nbqbits
reference_ket, hf_init_sp = molecule_factory.get_reference_ket(hf_init, nbqbits, transform)



print('Pool size: ', pool_size)
print('length of the cluster OP: ', len(cluster_ops))
print('length of the cluster OPS: ', len(cluster_ops_sp))
FCI = info['FCI']
# print(hf_init_sp)

 --------------------------------------------------------------------------
Running in the active case: 
                     molecule symbol: H4 
                     molecule basis: sto-3g 
                     type of generator: sUPCCGSD 
                     transform: JW 
 --------------------------------------------------------------------------
 --------------------------------------------------------------------------
                                                          
                      Generate Hamiltonians and Properties from :
                                                          
 --------------------------------------------------------------------------
                                                          
Number of electrons =  4
Number of qubits before active space selection =  8
Orbital energies =  [-0.70984573 -0.40884769  0.3923348   1.16930557]
Nuclear repulsion =  2.697766173317647
Noons =  [1.9815824741190093, 1.9433339987337899, 0.0605480741345

/home/haidora7789/OpenVQE/python/cleaned_code/UCC_active_space_selection/molecule_factory.py:465: UserWarning: This guess_init_state function is deprecated.
  n_elec, noons_full, orb_energies_full, hamiltonian.hpqrs


In [4]:
nbqbits = hamiltonian_sp.nbqbits
pool_type = 'without_Z_from_generator'
qubit_pool = qubitpool.generate_pool(cluster_ops)
len_returned_pool, returned_pool = qubitpool.generate_pool_without_cluster(pool_type=pool_type, 
                                                                        nbqbits=nbqbits, 
                                                                        qubit_pool=qubit_pool,
                                                                        molecule_symbol=
                                                                        molecule_symbol)

# pool_condition='full_without_Z'
# len_returned_pool, returned_pool = qubitpool.generate_hamiltonian_from_cluster(pool_condition, cluster_ops, nbqbits)
    

The current pool is without_Z_from_generator


In [5]:
# theta_current1 = theta_MP2
# theta_current2 = theta_MP2
# print(theta_current1)
print(len(cluster_ops_sp))
pool_generator = returned_pool
theta_current1 = []
theta_current2 = []
ansatz_ops = []
ansatz_q_ops = []
for i in range(len(cluster_ops_sp)):
    theta_current1.append(0.01)

for i in range(len(returned_pool)):
    theta_current2.append(0.01)
# for UCCS skip 1j
for i in cluster_ops_sp:
    ansatz_ops.append(i*1j)
for i in pool_generator:
    ansatz_q_ops.append(i)
iterations, result = energy_ucc.get_energies(hamiltonian_sp,ansatz_ops,ansatz_q_ops,hf_init_sp,theta_current1,theta_current2,FCI)
print("iterations are:", iterations)
print("results are:", result)

6
tolerance=  0.0001
method=  SLSQP
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1500542924513835
            Iterations: 3
            Function evaluations: 25
            Gradient evaluations: 3
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1499748626306383
            Iterations: 3
            Function evaluations: 23
            Gradient evaluations: 3
iterations are: {'minimum_energy_result1_guess': [-2.1500542924513835], 'minimum_energy_result2_guess': [-2.1499748626306383], 'theta_optimized_result1': [[-0.0013597570622389137, 0.005350006176654091, 0.010176423956701824, -0.1417612778394864, -0.03267633201353982, 0.003722342650564466]], 'theta_optimized_result2': [[0.0005848717116957872, -0.00206351711992418, 0.010092859630179205, -0.14069427567344203, -0.032850830196679295, 0.003704668557825702]]}
results are: {'CNOT1': 208, 'CNOT2': 168, 'len_op1': 6, 'len_op2': 6, 'energies1_substract

In [ ]:
print(iterations, result)